# Transformations
> TBD (transforming vectors and graphics)

## Contents
TBD

## Intro

In this chapter, and leaning on the concepts we've learned in the previous chapters, we'll see how we can render any 2D or 3D figure by decomposing it into line segments and polygons defined by vectors.

Also, we'll introduce the concept of animation, which consists in render slightly different static images in quick sequence to *cheat* or mind and make it think that the image is continuously changing.

This changes in the images are powered by transformations. As a mental model, you can think of transformation as black boxes that receive an input and produce a geometric change on the given input (rotate, translate, scale,...).

We will discuss a broad class of vector transformations called *linear transformations* that, like rotations, transform vectors on a straight line to new vectors that also lie on a straight line. *Linear transformations* have numerous applications in math, physics and data analysis.

To visualize rotations, linear transformations and other relevant transformations, *OpenGL* will be used using *PyOpenGL* library. For animations we'll use *PyGame* a game development library for Python.

## Transforming 3D objects

The goal of this chapter is to learn how to create new 3D objexts from existing ones applying transformations.

Every transformation we look at will take a vector as an input and will return another vector:

```python
def transform(v):
    old_x, old_y, old_z = v
    # ... transform ...
    return (new_x, new_y, new_z)
```

### Drawing a Transformed Object

We'll use the *Utah teapot* as our 3D model reference. We will modify the vectors defining the teapot and re-render it, so that we can see the geometric effect of the transformation.

When we run it without changes we see:

In [6]:
from draw_teapot import draw_teapot

draw_teapot()

SystemExit: 0

As an example, we can define a function `scale2(vector)` that multiplies any input vector by `2.0` and returns the result:

In [7]:
from vectors import scale

def scale2(v):
    return scale(2.0, v)

Note that this function complies with the form given for the `transform(...)` function above, and therefore, it is a transformation.

Let's apply it to the teapot vectors and see what happens:

In [8]:
from teapot import load_triangles
from draw_model import draw_model

original_triangles = load_triangles()
scaled_triangles = [ 
    [scale2(vertex) for vertex in triangle] 
    for triangle in original_triangles
]

draw_model(scaled_triangles)

SystemExit: 0

Let's now apply another simple transformation: translation. We want to move every vertex of the 3D model one unit in the negative x direction, apply to the already scaled figure and see what happens:

In [9]:
from vectors import add

def translate_1_left(v):
    return add((-1, 0, 0), v)

translated_scaled_triangles = [
    [ translate_1_left(vertex) for vertex in triangle ]
    for triangle in scaled_triangles
]

draw_model(translated_scaled_triangles)

SystemExit: 0

### Composing vector transformations

Applying any number of transformations sequentially defines a new transformation. For example, we could package what we did in 2 steps in the previous section into a single transformation:

In [10]:
def scale2_then_translate1left(v):
    return translate_1_left(scale2(v))

original_triangles = load_triangles()
transformed_triangles = [
    [scale2_then_translate1left(vertex) for vertex in triangle]
    for triangle in original_triangles
]

draw_model(transformed_triangles)

SystemExit: 0

As vector transformations take vectors and return vectors, we can combine any number of them using *function composition*:

```python
def compose(f1, f2):
    def new_function(input):
        return f1(f2(input))
    return new_function
```

In [11]:
def compose(f1, f2):
    def new_function(input):
        return f1(f2(input))
    return new_function

scale2_then_translate1left = compose(translate_1_left, scale2)

original_triangles = load_triangles()
transformed_triangles = [
    [scale2_then_translate1left(vertex) for vertex in triangle]
    for triangle in original_triangles
]

draw_model(transformed_triangles)

SystemExit: 0

We've already seen that having to apply the *list comprehension* technique each time that we want to apply a transformation is kind of a nuisance.

We can improve that by creating a helper function `polygon_map(transformation, polygons)`:

In [12]:
def polygon_map(transformation, polygons):
    return [
        [transformation(vertex) for vertex in polygon]
        for polygon in polygons
    ]

draw_model(
    polygon_map(
        compose(translate_1_left, scale2), load_triangles()
        )
    )

SystemExit: 0

Both `compose(...)` and `polygon_map(...)` take vector transformations as arguments. 

Sometimes, it's also useful to have functions that return transformations. For example, we can define a `scale_by(scalar)` function:

In [16]:
def scale_by(scalar):
    def new_function(v):
        return scale(scalar, v)
    return new_function

draw_model(polygon_map(scale_by(0.5), load_triangles()))

SystemExit: 0

Similarly, we can use the same approach (having functions that return transformations) to define a `translate_by(...)` function:

In [13]:
def translate_by(translation_vector):
    def new_function(v):
        return add(translation_vector, v)
    return new_function

draw_model(polygon_map(translate_by((-1, 0, 0)), load_triangles()))

SystemExit: 0

As a result, we now have a very flexible, *functional* way of applying transformations thanks to *composition* and *functions that return functions*:

In [14]:
draw_model(polygon_map(compose(translate_by((-1, 0, 0)), scale_by(2)), load_triangles()))

NameError: name 'scale_by' is not defined

### Rotating an object about an axis

We already learn how to do rotations in 2D in the chapter [01 &mdash; Vectors in the 2D plane](../../01-vectors-in-the-2d-plane).
The technique consisted in converting the Cartesian coordinates of the vector to be rotated to its polar coordinates, applying the rotation, and then convert it back to its Cartesian coordinates.

This technique is also useful for 3D rotations because all 3D vector rotations are *isolated in planes*.

For example, considered the following situation in which we want to rotate a point in the 3D space around the z-axis:

![3D space rotations](../images/3d-space-rotations.png)



Note that when the point is rotated, the z coordinate of the point does not change &mdash; it is isolated on the z plane, and therefore, we could apply the 2D rotation technique to the x and y coordinates of the point.

Let's define the rotation function:

In [15]:
from vectors import to_polar, to_cartesian

def rotate2d(angle, vector):
    vector_length, vector_angle = to_polar(vector)
    vector_angle = vector_angle + angle
    return to_cartesian((vector_length, vector_angle))

And now the 3D rotation function to compute rotations about the z-axis:

In [16]:
def rotate_z(angle, vector):
    v_x, v_y, v_z = vector
    new_x, new_y = rotate2d(angle, (v_x, v_y))
    return (new_x, new_y, v_z)

And if we define a `rotate_z_by(...)` function that returns a function that performs the transformation, we will be able to start using it for our renders:

In [18]:
def rotate_z_by(angle):
    def new_function(v):
        return rotate_z(angle, v)
    return new_function

In [19]:
from math import pi

draw_model(polygon_map(rotate_z_by(pi / 4.0), load_triangles()))

SystemExit: 0

We can write similar functions to rotate about the x and y axis:

In [20]:
def rotate_x(angle, vector):
    x, y, z = vector
    new_y, new_z = rotate2d(angle, (y, z))
    return (x, new_y, new_z)

def rotate_x_by(angle):
    def new_function(v):
        return rotate_x(angle, v)
    return new_function


draw_model(polygon_map(rotate_x_by(pi / 2.), load_triangles()))

SystemExit: 0

In [21]:
def rotate_y(angle, vector):
    x, y, z = vector
    new_x, new_z = rotate2d(angle, (x, z))
    return (new_x, y, new_z)

def rotate_y_by(angle):
    def new_function(v):
        return rotate_y(angle, v)
    return new_function

draw_model(polygon_map(rotate_y_by(pi / 4.), load_triangles()))

SystemExit: 0

### Inventing your own geometric transformations

We've seen so far the more common *transformations*: scaling, translation and rotation. However, we can define additional ones as long as they conform with the requirement:
> a transformation takes as input a 3D vector and return a 3D vector.

For example, we can define a `stretch_x(...)` that returns a vector whose x coordinate has been stretched:

In [24]:
def stretch_x(v):
    x, y, z = v
    return (4. * x, y, z)

draw_model(polygon_map(stretch_x, load_triangles()))

SystemExit: 0

In [25]:
def stretch_y(v):
    x, y, z = v
    return (x, 4. * y, z)

draw_model(polygon_map(stretch_y, load_triangles()))

SystemExit: 0

In [26]:
def cube_stretch_z(v):
    x, y, z = v
    return (x, y, z * z * z)

draw_model(polygon_map(cube_stretch_z, load_triangles()))

SystemExit: 0

In [ ]:
def cube_stretch_y(v):
    x, y, z = v
    return (x, y * y * y, z)

draw_model(polygon_map(cube_stretch_y, load_triangles()))

In [ ]:
Additional transformatios are possible, like *slanting*:

In [28]:
def slant_xy(v):
    x, y, z = v
    return (x + y, y, z)

draw_model(polygon_map(slant_xy, load_triangles()))

SystemExit: 0

## Linear Transformations

Linear transformations are special transformations where vector arithmetics looks the same before and after the transformation. Linear transformations are one of the main objects of study in linear algebra.

### Preserving vector arithmetic

Let's try to understand what *linear transformations* mean with examples: